In [1]:
import timm
from PIL import Image
import torch
from torchvision import transforms
from torch.nn import functional as F
import timm.models.mlp_mixer
import numpy as np
import exchange_tensor_array as exchange
import matplotlib.pyplot as plt

from pathlib import Path
from torchvision.datasets.utils import download_url
import json

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#モデル作成
model = timm.create_model("gmlp_s16_224", pretrained=True)
model.eval()
print("model")

model


In [3]:
transform = transforms.Compose(
    [
        transforms.Resize(256),  # (256, 256) で切り抜く。
        transforms.CenterCrop(224),  # 画像の中心に合わせて、(224, 224) で切り抜く
        transforms.ToTensor(),  # テンソルにする。
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # 標準化する。
    ]
)

In [4]:
img = Image.open("cat.jpg")
print(type(img))
inputs = transform(img)
inputs = inputs.unsqueeze(0)
print(type(inputs))

<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'torch.Tensor'>


In [5]:
inputs.shape

torch.Size([1, 3, 224, 224])

In [ ]:
for i in range(1000):
    output = model(inputs)

In [6]:
import PixelAblationCAMfrogMLP
exp = PixelAblationCAMfrogMLP.PixelAblationCAMfrogMLP(model)

In [ ]:
print(inputs.shape)

In [7]:
exp_out = exp.calc_value_las_block_beta(inputs)
print(exp_out)

KeyboardInterrupt: 

In [ ]:
print(exp_out[0])

In [ ]:
#推論
output = model(inputs)

In [ ]:
batch_probs = F.softmax(output, dim = 1)
batch_probs, batch_indices = batch_probs.sort(dim = 1, descending=True)
print(batch_probs[0][0])
print(type(batch_probs), type(batch_indices))
print(batch_indices[0][0])
def get_classes():
    if not Path("data/imagenet_class_index.json").exists():
        # ファイルが存在しない場合はダウンロードする。
        download_url("https://git.io/JebAs", "data", "imagenet_class_index.json")

    # クラス一覧を読み込む。
    with open("data/imagenet_class_index.json") as f:
        data = json.load(f)
        class_names = [x["ja"] for x in data]

    return class_names


# クラス名一覧を取得する。
class_names = get_classes()

In [ ]:
for probs, indices in zip(batch_probs, batch_indices):
    for k in range(3):
        print(f"Top-{k + 1} {class_names[indices[k]]} {probs[k]:.2%}")

In [ ]:
bp = batch_probs[0][0]
bi = batch_indices[0][0]
k = 0
print(f"Top-{k + 1} {class_names[bi]} {bp:.2%}")


In [ ]:
#print(class_names)

In [ ]:
tmp = model.tmp
tmp.shape

In [ ]:
bi = model.blocks_input
bo = model.blocks_output
print(torch.allclose(bi, tmp))
print(torch.allclose(bo, tmp))

In [ ]:
block_inputs = []
block_outputs = []
#ブロックの入出力の抽出
for i in model.blocks:
    _input = i.block_input
    #print(_input)
    #print(type(i))
    #print(vars(i).keys())
    _output = i.block_output
    block_inputs.append(_input)
    block_outputs.append(_output)
    print(torch.allclose(bi, _input))
    print(torch.allclose(bi, _output))
    print(torch.allclose(bo, _input))
    print(torch.allclose(bo, _output))
    #print(type(_input), _input.shape)
    #print(type(_output), _output.shape)

In [ ]:
for i in block_inputs:
    #print(i.shape == tmp.shape)
    print(torch.allclose(i, tmp))

In [ ]:
print(model.pooling_only(tmp) == output)

In [ ]:
b1 = block_outputs[0]
print(b1.shape)
p = exchange.exchange_tensor_to_array(i)

In [ ]:
j = 1
for i in block_outputs:
    p = exchange.exchange_tensor_to_array(i)
    #print(type(i[0]), i[0].shape)
    #print(type(tmp))
    #exchange.show_heatmap_with_colorbar(tmp)
    #print(type(tmp), tmp.shape)
    tmp_output = model.pooling_only(i)
    #print(tmp_output.shape)
    #print(tmp_output == output)
    #print(type(tmp_output), tmp_output.shape)
    if torch.allclose(tmp_output, output):
        print("A")
        print(j)
    j += 1

In [ ]:
#print(tmp_output == output)
print(torch.allclose(tmp_output, output))
tmp_output = model.pooling_only(block_outputs[-1])
print(torch.allclose(tmp_output, output))

In [ ]:
for i in block_outputs:
    break
    tmp = exchange.exchange_tensor_to_array(i[0])
    #print(type(tmp))
    exchange.show_heatmap_with_colorbar(tmp)